In [1]:
import pmx, pmx.forcefield
#import MDAnalysis as mda

In [2]:
base_dir = '/coarse/josef/ecc_lipids/topologies/'
itp_reorder_fname = base_dir+"lipid17/lipid17-POPG_amber-atomic-names_and_order_palmit-renamed.itp"
script_mapping_dir = base_dir+"../scripts/atomic_partial_charges_compar_plot_l14_slipids_C36/"
itp_reference_fname = base_dir+"Charmm36/POPG.itp"

In [3]:
# ITP to be reordered
top_reorder = pmx.forcefield.ITPFile(fname=itp_reorder_fname)
# converts atom indices (as read from ITP file) 
# into corresponding Atom instances
# in all angles, dihedrals, bonds, vsites...
top_reorder.id2atoms()

# ITP of the reference (has correct ordering)
top_reference = pmx.forcefield.ITPFile(fname=itp_reference_fname)

In [4]:
# Directly taken from the script that compares partial charges of various POPC models

def fetch_atom(mol, at_name):
    """
    return the first occurence of an atom with a given name.
    return the 1st atom if searched atom is not found.
    """
    try:
        found = False
        for otherat in mol.atoms:
            if otherat.name == at_name:
                found = True
                #print "Found atom {atname}.".format(atname=at_name)
                return otherat
    except:
        print "Something went wrong during atom searching (function fetch_atom). \
        \nIs mol a pmx.forcefield.ITPfile object?"
    finally:
        if not found:
            print "Atom {atname} not found! -- will substitute it with atom no.1.".format(atname=at_name)
            # so that there's no missing space in the sequence.
            return mol.atoms[0]



Beware!!
---
Lipid14 convention of naming atoms is different from any other model. It contains different atoms with same names -- the difference is made by assigning them a different residue name: [PC, palmitoyl, oleoyl].

mapping_ab is hence ambiguous if the chains palmitoyl--oleoyl are not differenciated!

Here I use a special differenciated version of atom naming convention that append letter "p" to every atom in palmitoyl chaing beginning with glycerol C1 carbon (carbon the palmitoyl is attached to).

In [5]:
# Directly taken from the script that compares partial charges of various POPC models
# with only minor modifications

# translating A->B, so
# B is a reference being a dictionary in order  FFname - Mapping_name
# whereas A is being translated so a dictionary in opposite order is more practical
# A: Mapping_name -> FFname
# mappingPOPClipid14.txt -- works only for lipid14
mapping_xa = {}
with open(script_mapping_dir+"mappingPOPClipid14_palmit.txt","r") as f:
    for line in f.readlines():
        if not line.startswith("#"):
            items = line.split()
            mapping_xa[items[0]] = items[1]

# mappingFILE.txt -- works for slipids & charmm
mapping_bx = {}
with open(script_mapping_dir+"mappingFILE.txt","r") as f:
    for line in f.readlines():
        if not line.startswith("#"):
            items = line.split()
            mapping_bx[items[1]] = items[0]

# create dictionaries for transaltion from a->b
mapping_ab = {}
for key in mapping_bx.keys():
    mapping_ab[mapping_xa[mapping_bx[key]]] = key

mapping_ba = {}
for key in mapping_bx.keys():
    mapping_ba[key] = mapping_xa[mapping_bx[key]]


This is and adaptation to DPPC of a previous ipython notebook for POPC

so I have to add the extra hydrogen at the middle palmitoyl chain (sn-2) at its very end (carbon 16, C216)
and also add the extra hydrogens at the former double bond (oleoyl -> palmitoyl)

Sorting didn't end up well -- check for errors and correct!!
==

In [6]:
# This was for DPPC:
#mapping_ab["H16T"] = "H16T"
#mapping_ab["H10R"] = "H10R"
#mapping_ab["H10S"] = "H10S"
#mapping_ab["H9R"] = "H9R"
#mapping_ab["H9S"] = "H9S"

# POPE:
mapping_ab["HN1A"] = "HN1"
mapping_ab["HN1B"] = "HN2"
mapping_ab["HN1C"] = "HN3"

# POPS:
# HR, HS belong to C1p in lipid17, but to C3 in charmm36 -- this was a mistake in an earlier version of this script!
mapping_ab["HR"] = "HX"
mapping_ab["HS"] = "HY"
mapping_ab["C33"] = "C13"
#mapping_ab["O35"] = "O13A"
#mapping_ab["O36"] = "O13B"

# POPG:
mapping_ab["HO5A"] = "HO2"
mapping_ab["HO6A"] = "HO3"
mapping_ab["C31"] = "C11"
mapping_ab["C32"] = "C12"
mapping_ab["C33"] = "C13"
mapping_ab["H3A"] = "H13A"
mapping_ab["H3B"] = "H13B"
mapping_ab["H1A"] = "H11A"
mapping_ab["H1B"] = "H11B"
mapping_ab["H2A"] = "H12A"
mapping_ab["O35"] = "OC2"
mapping_ab["O36"] = "OC3"
#mapping_ab[""] = ""
#mapping_ab[""] = ""
    



In [7]:
for a in top_reorder.atoms:
    a.resname = 'POPG'

Simple solution: Converting conventions of atom naming using mapping files from Lipid14 -> Charmm36 convention

In [8]:
# a simple solution to the problem using mapping_ab can be used 
# only after the differentiation of atom names 
# between palmitoyl/oleoyl chains in LIPID14 nomenclature
top_reorder.atoms.sort(key=lambda a: fetch_atom(top_reference, mapping_ab[a.name]).id)
for i,a in enumerate(top_reorder.atoms):
    a.id = i+1

In [9]:
top_reorder.write(base_dir+"lipid17/lipid17-POPG_charmm36-reordered.itp")

Sorting the atoms of ECC-POPC according to the ordering of Charmm36 POPC

In [10]:
top_reorder.atoms.sort(key=lambda a: a.id)

In [11]:
top_reorder.write(base_dir+"lipid17/lipid17-POPG_charmm36-reordered_sort.itp")

GOOD! both topologies are the same!

Change also the atom names, so that the POPC lipid really looks like CHARMM36 lipid!

In [12]:
# somewhat buggy renaming ???
#for a in top_reference.atoms:
#    #fetch_atom(top_reorder, mapping_ba[a.name]).id = a.id
#    fetch_atom(top_reorder, mapping_ba[a.name]).name = a.name

In [13]:
#top_reorder.atoms.sort(key=lambda a: a.id)

In [14]:
for i,a in enumerate(top_reference.atoms):
    top_reorder.atoms[i].name = a.name

In [15]:
top_reorder.write(base_dir+"lipid17/lipid17-POPG_charmm36-atomic-names.itp")

And now -- make a ECC-POPS by scaling it with 3/4!
==

In [16]:
# read the topology in again to correct for rounding errors
top_ecc = pmx.forcefield.ITPFile(base_dir+"lipid17/lipid17-POPG_charmm36-atomic-names.itp")

In [17]:
for a in top_ecc.atoms:
    # save the original charge in B-top field
    a.qB = a.q
    a.q *= 0.75
    # only for the head group - i.e. until atom H2Y
    if a.name=='H2Y': 
        print "scaled until atom id {}, which shall incorporate whole PG head group".format(a.id)
        break

scaled until atom id 37, which shall incorporate whole PG head group


In [18]:
qtot = 0.0
for a in top_ecc.atoms:
    qtot += a.q
    
print qtot

-0.72193675


In [19]:
print fetch_atom(top_ecc, "O13A").q
print fetch_atom(top_ecc, "O13B").q
print fetch_atom(top_ecc, "P").q

Atom O13A not found! -- will substitute it with atom no.1.
0.07291875
Atom O13B not found! -- will substitute it with atom no.1.
0.07291875
0.8624625


In [20]:
q_extra = qtot+0.75

for a in top_ecc.atoms:
    if a.name in ['P', ]:
        #a = fetch_atom(top_reference, atomname)
        # subtracting extra excess charge from Carboxyl oxygen atoms equally
        print a.name, a.q
        a.q -= q_extra

P 0.8624625


In [21]:
qtot = 0.0
for a in top_ecc.atoms:
    qtot += a.q
    
print qtot

-0.75


In [22]:
top_ecc.write(base_dir+"lipid17/ECC-POPG_charmm36-atomic-names.itp")

small manual edit of the resulting file
---
correcting for charge round-up errors with manual edits to O13x charges adding 1e-6 to each of them (two)

In [23]:
del(top_ecc)

In [24]:
# read the topology in again to correct for rounding errors
top_ecc = pmx.forcefield.ITPFile(base_dir+"lipid17/ECC-POPG_charmm36-atomic-names.itp")

In [25]:
qtot = 0.0
for a in top_ecc.atoms:
    qtot += a.q
    
print qtot

-0.749997
